In [1]:
!cp '/content/drive/MyDrive/HACKER EARTH/7342422066d511eb.zip'  .

In [2]:
!unzip -q 7342422066d511eb.zip

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

path = os.getcwd()
data_path = '//content//drive//MyDrive//HACKER EARTH//'
data_path1='//content//dataset//'

In [4]:
train = pd.read_csv(data_path + 'cln_train.csv')
test = pd.read_csv(data_path + 'cln_test.csv')
sample_sub = pd.read_csv(data_path1 + 'sample_submission.csv')

In [5]:
train[:5]

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,06/07/15,06/03/15,"New Michelle, OH 50777",-283.29
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,03/06/17,03/05/17,"New Michaelport, WY 12072",-159.96
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,03/09/15,03/08/15,"Bowmanshire, WA 19241",-154.29
3,fffe350031003300,Robert Chaires,0.12,9.0,8.0,3102.0,Aluminium,5.81,16.31,No,No,No,Roadways,No,Wealthy,Yes,05/24/15,05/20/15,"East Robyn, KY 86375",-161.16
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,12/18/16,12/14/16,"Aprilside, PA 52793",-159.23


In [6]:
sample_sub[:5]

,Customer Id,Cost
0,fffe3900350033003300,283.29
1,fffe3800330031003900,159.96
2,fffe3600370035003100,154.29
3,fffe350031003300,161.16
4,fffe3900320038003400,159.23


In [7]:
sample_sub = test[['Customer Id', 'Artist Name']].copy()
sample_sub['Cost'] = None
sample_sub = sample_sub.drop('Artist Name', axis = 1)

In [8]:
sample_sub[:5]

,Customer Id,Cost
0,fffe3400310033003300,None
1,fffe3600350035003400,None
2,fffe3700360030003500,None
3,fffe350038003600,None
4,fffe3500390032003500,None


In [9]:
train['Cost'] = train['Cost'].abs()

In [10]:
train['Cost'] = np.log(train['Cost'])

**Data Processing**

In [11]:
df = pd.concat([train, test], axis = 0).reset_index(drop=True)

In [12]:
ID = 'Customer Id'
name = 'Artist Name'
date_1 = 'Scheduled Date'
date_2 = 'Delivery Date'
location = 'Customer Location'
target = 'Cost'


cat_cols = ['Material','International', 'Express Shipment', 
            'Installation Included','Transport',
            'Fragile', 'Customer Information', 
            'Remote Location']

num_cols = ['Artist Reputation', 'Height', 'Width','Weight',
            'Price Of Sculpture', 'Base Shipping Price']

In [13]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [14]:
le = LabelEncoder()


In [15]:
df[cat_cols] = df[cat_cols].apply(le.fit_transform)

In [16]:
df[:5]

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,1,13.91,16.27,1,1,0,0,0,1,0,06/07/15,06/03/15,"New Michelle, OH 50777",5.646471
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,1,6.83,15.00,0,0,0,1,0,1,0,03/06/17,03/05/17,"New Michaelport, WY 12072",5.074924
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,3,4.96,21.18,0,0,0,1,1,1,1,03/09/15,03/08/15,"Bowmanshire, WA 19241",5.038834
3,fffe350031003300,Robert Chaires,0.12,9.0,8.0,3102.0,0,5.81,16.31,0,0,0,1,0,0,1,05/24/15,05/20/15,"East Robyn, KY 86375",5.082398
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,0,3.18,11.94,1,1,1,0,0,1,0,12/18/16,12/14/16,"Aprilside, PA 52793",5.070350


In [17]:
features = [col for col in df.columns if col not in ([ID, name, date_1, date_2, location, target])]

In [18]:
features[:5]

['Artist Reputation', 'Height', 'Width', 'Weight', 'Material']

In [19]:
train_proc, test_proc = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop = True)

In [20]:
train_proc[:5]

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,1,13.91,16.27,1,1,0,0,0,1,0,06/07/15,06/03/15,"New Michelle, OH 50777",5.646471
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,1,6.83,15.00,0,0,0,1,0,1,0,03/06/17,03/05/17,"New Michaelport, WY 12072",5.074924
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,3,4.96,21.18,0,0,0,1,1,1,1,03/09/15,03/08/15,"Bowmanshire, WA 19241",5.038834
3,fffe350031003300,Robert Chaires,0.12,9.0,8.0,3102.0,0,5.81,16.31,0,0,0,1,0,0,1,05/24/15,05/20/15,"East Robyn, KY 86375",5.082398
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,0,3.18,11.94,1,1,1,0,0,1,0,12/18/16,12/14/16,"Aprilside, PA 52793",5.070350


In [21]:
test_proc[:5]

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3400310033003300,James Miller,0.35,53.0,18.0,871.0,6,5.98,19.11,1,1,0,0,0,1,0,07/03/17,07/06/17,"Santoshaven, IA 63481",NaN
1,fffe3600350035003400,Karen Vetrano,0.67,7.0,4.0,108.0,3,6.92,13.96,0,0,0,1,1,1,0,05/02/16,05/02/16,"Ericksonton, OH 98253",NaN
2,fffe3700360030003500,Roseanne Gaona,0.61,6.0,5.0,97.0,0,4.23,13.62,1,0,0,0,0,1,0,01/04/18,01/06/18,APO AP 83453,NaN
3,fffe350038003600,Todd Almanza,0.14,15.0,8.0,757.0,3,6.28,23.79,0,1,0,1,1,0,0,09/14/17,09/17/17,"Antonioborough, AL 54778",NaN
4,fffe3500390032003500,Francis Rivero,0.63,10.0,4.0,1673.0,4,4.39,17.83,0,1,1,1,0,1,1,12/03/17,12/02/17,"Lake Frances, LA 03040",NaN


In [22]:
!pip install catboost

     |████████████████████████████████| 65.7MB 68kB/s 


In [23]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

In [24]:
trn, val = train_test_split(train_proc, test_size = 0.2, random_state = 1999)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test_proc[features]

In [25]:
model_dict = {}

model_dict['Linear Regression'] = LinearRegression()
model_dict['Decision Tree'] = DecisionTreeRegressor(random_state = 1)
model_dict['Extra Trees'] = ExtraTreesRegressor(random_state = 1)
model_dict['Random Forest'] = RandomForestRegressor(random_state = 1)
model_dict['AdaBoost'] = AdaBoostRegressor(random_state = 1)
model_dict['Bagging'] = BaggingRegressor(random_state = 1)
model_dict['XGBoost'] = XGBRegressor(random_state = 1)
model_dict['LGBM'] = LGBMRegressor(random_state = 1)
model_dict['Cat'] = CatBoostRegressor(verbose=False, random_state =1)

In [26]:
def model_test(X_train, X_test, y_train, y_test,model,model_name):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    y_pred = np.abs(y_pred)
    accuracy = 100*max(0, 1-mean_squared_error(y_test, y_pred))
    print('======================================{}======================================='.format(model_name))
    print('Score is : {}'.format(accuracy))
    print()

In [27]:
for model_name,model in model_dict.items():
    model_test(X_trn, X_val, y_trn, y_val, model, model_name)

======================================Linear Regression=======================================
Score is : 9.687567844968093

======================================Decision Tree=======================================
Score is : 76.30873543602085

======================================Extra Trees=======================================
Score is : 85.93373202721527

======================================Random Forest=======================================
Score is : 86.9351383257797

======================================AdaBoost=======================================
Score is : 62.97557060683279

======================================Bagging=======================================
Score is : 85.3034956605891

[03:24:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
======================================XGBoost=======================================
Score is : 84.89828197076767

======================================LGBM=

In [28]:
lgb_params = {'max_depth': 4, 'n_estimators': 1696, 'learning_rate': 0.0891, 'num_leaves': 318}

In [29]:
cat = CatBoostRegressor(random_state = 1)

_ = cat.fit(train_proc[features], train_proc[target])

preds = cat.predict(test_proc[features])
preds = np.exp(preds)

Learning rate set to 0.054463
0:	learn: 1.5782258	total: 3.19ms	remaining: 3.19s
1:	learn: 1.5073209	total: 6.25ms	remaining: 3.12s
2:	learn: 1.4405169	total: 11.1ms	remaining: 3.7s
3:	learn: 1.3793193	total: 14ms	remaining: 3.49s
4:	learn: 1.3223010	total: 16.6ms	remaining: 3.31s
5:	learn: 1.2661248	total: 19.4ms	remaining: 3.21s
6:	learn: 1.2150674	total: 21.9ms	remaining: 3.11s
7:	learn: 1.1644341	total: 24.8ms	remaining: 3.08s
8:	learn: 1.1178529	total: 27.8ms	remaining: 3.06s
9:	learn: 1.0749148	total: 30.6ms	remaining: 3.03s
10:	learn: 1.0313950	total: 33.3ms	remaining: 3s
11:	learn: 0.9947295	total: 36.1ms	remaining: 2.97s
12:	learn: 0.9586958	total: 38.8ms	remaining: 2.94s
13:	learn: 0.9241294	total: 41.5ms	remaining: 2.93s
14:	learn: 0.8908522	total: 44.3ms	remaining: 2.91s
15:	learn: 0.8603750	total: 47ms	remaining: 2.89s
16:	learn: 0.8309606	total: 49.7ms	remaining: 2.87s
17:	learn: 0.8042639	total: 52.3ms	remaining: 2.85s
18:	learn: 0.7778540	total: 54.9ms	remaining: 2.83s


In [30]:
sample_sub[target] = preds
sample_sub.to_csv(data_path1 + 'sub1.6.csv', index = False)

In [31]:
sample_sub

,Customer Id,Cost
0,fffe3400310033003300,287.062829
1,fffe3600350035003400,274.599095
2,fffe3700360030003500,196.693794
3,fffe350038003600,206.042259
4,fffe3500390032003500,312.484802
...,...,...
3495,fffe3200300037003100,1084.213839
3496,fffe390031003100,3362.941762
3497,fffe3200350031003700,675.220910
3498,fffe3100380031003000,261.583327


**Cross Validation technique**

In [32]:
# Cross Validation for Boosting
def cross_val(regressor, train, test, features, name):
    N_splits = 7
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
        if name != 'cat':
            #### Scaling Data ####
            scaler = StandardScaler()
            _ = scaler.fit(X_trn)
            X_trn = scaler.transform(X_trn)
            X_val = scaler.transform(X_val)
            X_test = scaler.transform(X_test)
        
        ############ Fitting #############
        _ = regressor.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 100, verbose = False)
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = 100*max(0, 1-mean_squared_error(y_val, val_preds))
        print(f'\n Score for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = 100*max(0, 1-mean_squared_error(y_val, val_preds))
    print(f'\n\Score for oofs is {total_error}')
    
    return oofs, preds

In [33]:
# Cross Validation for Boosting
def normal_cross_val(regressor, train, test, features, name):
    N_splits = 10
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    folds = StratifiedKFold(n_splits = N_splits, shuffle = True,random_state = 1999)
    stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    for index, (trn_idx, val_idx) in enumerate(folds.split(train, stratified_target)):
        print(f'\n================================Fold{index + 1}===================================')
        
        #### Train Set
        X_trn, y_trn = train[features].iloc[trn_idx], train[target].iloc[trn_idx]
        
        #### Validation Set
        X_val, y_val = train[features].iloc[val_idx], train[target].iloc[val_idx]
        
        #### Test Set
        X_test = test[features]
        
        if name != 'cat':
            #### Scaling Data ####
            scaler = StandardScaler()
            _ = scaler.fit(X_trn)
            X_trn = scaler.transform(X_trn)
            X_val = scaler.transform(X_val)
            X_test = scaler.transform(X_test)
        
        ############ Fitting #############
        _ = regressor.fit(X_trn, y_trn)
        
        ############ Predicting #############
        val_preds = np.abs(regressor.predict(X_val))
        test_preds = np.abs(regressor.predict(X_test))
        
        error = 100*max(0, 1-mean_squared_error(y_val, val_preds))
        print(f'\n Score for Validation set is : {error}')
        
        oofs[val_idx] = val_preds
        preds += test_preds / N_splits
        
    total_error = 100*max(0, 1-mean_squared_error(y_val, val_preds))
    print(f'\n\Score for oofs is {total_error}')
    
    return oofs, preds

In [34]:
params = {'depth': 5, 'n_estimators': 1703, 'learning_rate': 0.09193293802306104}
cat = CatBoostRegressor(random_state = 1, **params)

In [35]:
cat_oofs, cat_preds = cross_val(cat, train_proc, test_proc, features, 'cat')


================================Fold1===================================

 Score for Validation set is : 92.57061817612927

================================Fold2===================================

 Score for Validation set is : 93.62190541480824

================================Fold3===================================

 Score for Validation set is : 91.27106715266152

================================Fold4===================================

 Score for Validation set is : 91.13497297144396

================================Fold5===================================

 Score for Validation set is : 90.94660079021112

================================Fold6===================================

 Score for Validation set is : 91.10788376345646

================================Fold7===================================

 Score for Validation set is : 93.5666225031966

\Score for oofs is 93.5666225031966


In [36]:
sample_sub[target] = np.exp(cat_preds)
sample_sub.to_csv(data_path1 + 'sub1.4.csv', index = False)

In [37]:
sample_sub

,Customer Id,Cost
0,fffe3400310033003300,266.830044
1,fffe3600350035003400,270.867685
2,fffe3700360030003500,182.629499
3,fffe350038003600,210.983849
4,fffe3500390032003500,319.603942
...,...,...
3495,fffe3200300037003100,1102.848406
3496,fffe390031003100,3010.839798
3497,fffe3200350031003700,622.483423
3498,fffe3100380031003000,270.970609


In [38]:
!pip install optuna

     |████████████████████████████████| 296kB 4.2MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
     |████████████████████████████████| 163kB 29.0MB/s 
     |████████████████████████████████| 143kB 31.6MB/s 
     |████████████████████████████████| 112kB 31.6MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=33a9adb5f7b421c8fce56a7da443b70e39878c6a488c996f7d49eae83c27f8ed
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


**hyperparameter tunning**

In [42]:
import optuna 
from sklearn.metrics import *
from sklearn.model_selection import KFold 
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import numpy as np

def objective(trial): 
  param = { "max_depth": trial.suggest_int("max_depth", 3, 20), "n_estimators": trial.suggest_int("n_estimators", 500, 1500), "learning_rate": trial.suggest_float("learning_rate", 0.01, 1), "num_leaves" : trial.suggest_int("num_leaves", 40, 500)

  #"l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 6),
  #"subsample": trial.suggest_float("subsample", 0.1, 3),
  #'colsample_bylevel': trial.suggest_float('colsample_bylevel,', 0.1, 2),
  #'random_strength':trial.suggest_int('random_strength', 0, 1) 
        }
  skf = KFold(n_splits = 5, shuffle=True, random_state=42)

  for fold, (tr_ind, val_ind) in enumerate(skf.split(train_proc[features], train_proc[target])):

   X_train, X_val = train_proc[features].loc[tr_ind], train_proc[features].loc[val_ind]
   y_train, y_val = train_proc[target][tr_ind], train_proc[target][val_ind] 

   model = LGBMRegressor(**param, random_state=42)      

   model.fit(X_train, y_train) 

   val_pred  = np.abs(model.predict(X_val))
   accuracy  = 100 * max(0, 1-mean_squared_error(y_val, val_pred))

   return accuracy

study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=100)

[I 2021-03-20 03:28:53,565] A new study created in memory with name: no-name-9833fd7f-5067-477a-ac13-c41f62a59d4c
[I 2021-03-20 03:28:56,753] Trial 0 finished with value: 86.84729317311954 and parameters: {'max_depth': 9, 'n_estimators': 1316, 'learning_rate': 0.5656113552238982, 'num_leaves': 202}. Best is trial 0 with value: 86.84729317311954.
[I 2021-03-20 03:28:58,272] Trial 1 finished with value: 87.96396290824889 and parameters: {'max_depth': 5, 'n_estimators': 1421, 'learning_rate': 0.3982756886871995, 'num_leaves': 478}. Best is trial 1 with value: 87.96396290824889.
[I 2021-03-20 03:28:59,901] Trial 2 finished with value: 87.97720749057652 and parameters: {'max_depth': 15, 'n_estimators': 522, 'learning_rate': 0.2865721924058486, 'num_leaves': 235}. Best is trial 2 with value: 87.97720749057652.
[I 2021-03-20 03:29:04,859] Trial 3 finished with value: 89.26668004980179 and parameters: {'max_depth': 16, 'n_estimators': 1348, 'learning_rate': 0.018213093887507317, 'num_leaves': 

In [ ]:
import optuna 
from sklearn.metrics import *
from sklearn.model_selection import KFold 
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import numpy as np

def objective(trial): 
  param = {"depth": trial.suggest_int("depth", 3, 16), "n_estimators": trial.suggest_int("n_estimators", 5000, 15000), "learning_rate": trial.suggest_float("learning_rate", 0.01, 1)

  ,"l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 3),
  #"subsample": trial.suggest_float("subsample", 0.1, 3),
  #'colsample_bylevel': trial.suggest_float('colsample_bylevel,', 0.1, 2),
  #'random_strength':trial.suggest_int('random_strength', 0, 1) 
        }
   skf = KFold(n_splits = 5, shuffle=True, random_state=42)

   for fold, (tr_ind, val_ind) in enumerate(skf.split(train_proc[features], train_proc[target])):

      X_train, X_val = train_proc[features].loc[tr_ind], train_proc[features].loc[val_ind]
      y_train, y_val = train_proc[target][tr_ind], train_proc[target][val_ind] 

      model = CatBoostRegressor(**param, random_state=42)      

      model.fit(X_train, y_train) 

      val_pred  = np.abs(model.predict(X_val))
      accuracy  = 100 * max(0, 1-mean_squared_error(y_val, val_pred))

   return accuracy

study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=100)

[I 2021-02-27 20:25:44,382] A new study created in memory with name: no-name-101510ed-86c0-4337-a1a7-ce5818ed8795


0:	learn: 0.7217408	total: 333ms	remaining: 1h 10m 24s
1:	learn: 0.5305471	total: 1.55s	remaining: 2h 43m 43s
2:	learn: 0.4220900	total: 3s	remaining: 3h 31m 1s
3:	learn: 0.3598800	total: 4.52s	remaining: 3h 58m 52s
4:	learn: 0.3217109	total: 6s	remaining: 4h 13m 20s
5:	learn: 0.2841634	total: 7.54s	remaining: 4h 25m 30s
6:	learn: 0.2562460	total: 9.3s	remaining: 4h 40m 27s
7:	learn: 0.2384783	total: 10.9s	remaining: 4h 47m 12s
8:	learn: 0.2234404	total: 12.5s	remaining: 4h 52m 36s
9:	learn: 0.2033197	total: 14s	remaining: 4h 55m 42s
10:	learn: 0.1881370	total: 15.4s	remaining: 4h 56m 22s
11:	learn: 0.1711791	total: 16.9s	remaining: 4h 57m 35s
12:	learn: 0.1558553	total: 18.5s	remaining: 4h 59m 58s
13:	learn: 0.1464795	total: 20s	remaining: 5h 1m 51s
14:	learn: 0.1344863	total: 21.2s	remaining: 4h 58m 25s
15:	learn: 0.1295161	total: 21.4s	remaining: 4h 42m 46s
16:	learn: 0.1200533	total: 22.3s	remaining: 4h 36m 46s
17:	learn: 0.1131174	total: 23.2s	remaining: 4h 31m 24s
18:	learn: 0.10

KeyboardInterrupt: ignored